# Assignment 5 · Revisiting HHL for a 4×4 Linear System
This notebook is a code-completion exercise. Work through the TODO placeholders to rebuild the Harrow–Hassidim–Lloyd (HHL) workflow, compare it with a classical baseline, and analyse the tomography surrogate.

This refreshed notebook mirrors the structured workflow from Assignment 3 so every phase stays auditable and reproducible.

**How to proceed**
1. Advance task by task, filling the TODO markers inside the code cells.
2. Keep intermediate calculations visible so mentors can review reasoning.
3. Reuse utilities from earlier assignments where prompted (e.g., the QST surrogate).
4. Document any modelling choices directly in the notebook markdown.

## Background notes
- HHL targets systems $A\vec{x}=\vec{b}$ where $A$ is Hermitian and sized $2^n \times 2^n$, embedding the matrix into a unitary, applying phase estimation, and inverting eigenvalues by a controlled rotation.
- We choose a modest 4×4 Hermitian matrix with a friendly condition number so that simulation resources focus on algorithmic steps rather than numerical instability.
- Diagnostics include component-wise differences, the $\ell_2$ vector error, and the residual norm $\lVert A\vec{x}_{\text{est}}-\vec{b}\rVert_2$ to keep classical and quantum results comparable.

## Task 1 · Environment setup
- Confirm the required Python packages are installed.
- Use the provided cells to record package versions once installation is complete.

In [ ]:
!pip install qiskit 

In [ ]:
# TODO: Update the package list if additional dependencies are required for your solution.
import importlib.metadata as metadata

packages = ["qiskit", "qiskit-algorithms", "numpy", "scipy", "pandas", "matplotlib"]
for name in packages:
    try:
        print(f"{name}: {metadata.version(name)}")
    except metadata.PackageNotFoundError:
        print(f"{name}: not installed")

## Task 2 · Specify the linear system
- Complete the TODOs to define a 4×4 Hermitian matrix `A` and a right-hand-side vector `b`.
- Compute classical diagnostics, normalised vectors, and store results in data structures for later comparison.

In [ ]:
import numpy as np
import pandas as pd

def prepare_linear_system() -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, pd.DataFrame, dict]:
    """
    TODO: Specify the Hermitian matrix A and right-hand side b for the 4×4 system.

    Implement the following steps inside this helper:
    1. Define a well-conditioned Hermitian matrix `A` of shape (4, 4).
    2. Define the complex right-hand-side vector `b` and normalise it to obtain `b_norm`.
    3. Solve the system classically to obtain `x_classical` and its normalised form.
    4. Compute diagnostic quantities (eigenvalues, condition number, residuals, norms).
    5. Package the component-wise data in a pandas DataFrame and the diagnostics in a dict.

    Returns:
        A tuple containing `(A, b, b_norm, x_classical, x_classical_norm, system_df, diagnostics)`.
    """
    raise NotImplementedError("Specify the linear system and classical diagnostics.")

A, b, b_norm, x_classical, x_classical_norm, system_df, diagnostics = prepare_linear_system()
system_df, diagnostics

## Task 3 · Implement the HHL solver
- Fill in the helper that builds and executes HHL using Qiskit primitives.
- Extract the solution register, normalise the amplitudes, and return artefacts needed for downstream analysis.

In [ ]:
from qiskit.quantum_info import Statevector
import numpy as np
import pandas as pd

def run_hhl_and_extract(A: np.ndarray, b_normalised: np.ndarray):
    """
    TODO: Implement the HHL algorithm to solve the linear system Ax=b.

    This function should:
    1. Define a `LinearSystemProblem`.
    2. Instantiate and run the `HHL` solver.
    3. Extract the full statevector from the result.
    4. Isolate the raw solution amplitudes from the appropriate register.
    5. Normalise the solution vector.
    6. Return the raw solution, normalised solution, full statevector, and the HHL result object.
    """
    pass


In [ ]:
def summarise_hhl_solution(A: np.ndarray, b: np.ndarray, b_norm: np.ndarray, x_classical: np.ndarray):
    """
    TODO: Execute HHL, align the quantum solution with the classical baseline, and compute diagnostics.

    Implement the following steps:
    1. Call `run_hhl_and_extract` to obtain the raw and normalised HHL amplitudes plus the full statevector.
    2. Align the global phase and amplitude against `x_classical` to obtain `x_hhl_rescaled`.
    3. Compute the L2 error, relative error, residual norm, and scale factor.
    4. Build a pandas DataFrame comparing component-wise amplitudes and absolute errors.
    5. Package scalar diagnostics in a dictionary.

    Returns:
        comparison_df, metrics, raw_hhl, norm_hhl, full_statevector, hhl_result
    """
    raise NotImplementedError("Implement the HHL vs classical comparison workflow.")

comparison_df, metrics, raw_hhl, norm_hhl, full_statevector, hhl_result = summarise_hhl_solution(A, b, b_norm, x_classical)
comparison_df, metrics

## Task 4 · Execute HHL and compare solutions
- Run the HHL solver on the prepared linear system.
- Align the quantum output with the classical solution, then tabulate component-wise errors and aggregated metrics.

## Task 5 · Tomography cross-check with the ML surrogate
- Reuse the quantum state tomography (QST) regression model from Assignment 3 to rebuild the HHL solution from synthetic measurement statistics.
- Generate Pauli-basis expectation values from the HHL statevector, feed them through the surrogate, and recover an estimated statevector.
- Compare the surrogate reconstruction with both the raw HHL amplitudes and the classical baseline to quantify reconstruction accuracy.

**What to do**
- Instantiate the Pauli-basis surrogate, compute expectation values for every measurement setting, and reconstruct the density matrix.
- Extract the principal eigenstate, fix global phase, and report fidelities plus residuals alongside both baselines.

In [ ]:
from pathlib import Path
import pickle

model_path = "models/qst.pkl"
model = None
path = Path(model_path)
if not path.exists():
    raise FileNotFoundError(f"Tomography surrogate not found at {model_path}")

with path.open("rb") as fh:
    model = pickle.load(fh)

In [ ]:
def analyse_tomography_surrogate(full_statevector, norm_hhl, x_classical, scale_factor, A, b):
    """
    TODO: Reconstruct the HHL solution via tomography and compare with baselines.

    Implement the following steps:
    1. Instantiate `PauliTomographyModel` with the appropriate number of solution qubits.
    2. Extract the solution register from `full_statevector` to obtain the statevector analysed by tomography.
    3. Generate Pauli expectation values, reconstruct the density matrix, and estimate the dominant eigenstate.
    4. Align the global phase with the HHL reference and rescale using `scale_factor`.
    5. Compute comparison metrics and create a DataFrame similar to the HHL vs classical table.
    6. Return the DataFrame and a dictionary of tomography diagnostics (fidelities, errors, residual).
    """
    raise NotImplementedError("Implement the tomography cross-check once the surrogate is ready.")

scale_factor = metrics.get("scale_factor") if isinstance(metrics, dict) else None
comparison_qst_df, tomography_metrics = analyse_tomography_surrogate(full_statevector, norm_hhl, x_classical, scale_factor, A, b)
comparison_qst_df, tomography_metrics

### Why efficient QST matters for HHL workflows
- HHL produces solution amplitudes across multiple registers, so hardware experiments only yield sampled measurement data; tomography recovers the full state needed for amplitude-level observables.
- Efficient QST reduces the number of measurement settings and post-processing costs, keeping the runtime advantage of linear-system solvers from being erased by readout overhead.
- ML-based surrogates let us amortise reconstruction across many runs (e.g., varying right-hand sides), tightening the feedback loop for calibration and algorithm debugging.

## Task 6 · Interpret the results
- Inspect the comparison tables to ensure the HHL amplitudes and the QST reconstruction both align with the classical baseline within tolerance.
- Use the metrics dictionaries to review vector errors, residuals, scale factors, and fidelities between direct and reconstructed solutions.

**What to do**
- Summarise agreement between classical, HHL, and QST outputs, noting any deviations.
- Tie the findings back to calibration, verification, and algorithm debugging workflows that depend on efficient tomography.

## Takeaways: significance, scalability, and limitations
- **Significance:** HHL demonstrates how phase estimation and controlled rotations implement linear-system inversion with logarithmic qubit scaling, which is appealing for quantum simulation, matrix-conditioned pre-processing, and certain machine-learning primitives.
- **Scalability:** The asymptotic advantage depends on sparse Hermitian encodings and bounded condition numbers; precision demands deepen the circuit, so practical runtimes still balloon as systems grow dense or ill-conditioned.
- **Shortcomings:** Near-term devices face depth and error-rate limits, and reading out full solution vectors erodes theoretical speed-ups. Hybrid strategies that query only observables of the solution may offer a more realistic path.
- **Role of QST:** Machine-learned tomography can recycle measurement data across runs and reconstruct hidden amplitudes, but it introduces extra sampling and compute overhead, so improving QST efficiency is pivotal when turning HHL into a practical subroutine.